In [109]:
# importing libraries
import pandas as pd
%matplotlib inline
import spacy
from spacy.attrs import LOWER, POS, ENT_TYPE, IS_ALPHA
from spacy.tokens import Doc
import numpy
from spacy.lang.en import STOP_WORDS

In [110]:
# loading the spacy model for english
nlp = spacy.load('en')

# loading the data set
data = pd.read_csv('../input/train (3) (1) (3) (2).csv')

# creating a list of stop-words
stop = list(STOP_WORDS)

In [111]:
# removing columns that are not used
data_new = data.drop(['App Version Code', 'App Version Name', 'Review Title'], axis = 1)
data_new = data_new.dropna(axis=0)

In [239]:
# tokenising and cleaning the words from extra elements
def word_clean(doc):
    doc = doc.lower()
    doc = nlp(doc)
    words = []
    for word in doc:
        if word.pos_ not in ('PUNCT', 'NUM', 'SYM'):
            words.append(word.lemma_)
    
    return words

In [240]:

data_new['Review Text'] = data_new['Review Text'].apply(word_clean)

AttributeError: 'list' object has no attribute 'lower'

In [241]:
# creating a bag of words
BOW = []
for rev in data_new['Review Text']:
    for word in rev:
        if word not in BOW:
            BOW.append(word)

In [242]:
bow_df = pd.DataFrame(columns=BOW, index=data_new['id'])
bow_df = bow_df.fillna(0)

In [243]:
for i, doc in enumerate(data_new['Review Text']):
    for c in bow_df.columns:
        if c in doc:
            bow_df[c][i] = 1

In [244]:
bow_df_2 = bow_df
bow_df.shape

(5692, 4533)

In [ ]:
s = 0
for c in bow_df.columns:
    if bow_df[c].sum() < 50:
        bow_df_2 = bow_df_2.drop(c, axis = 1)

In [ ]:
bow_df_2.shape

In [ ]:
X = bow_df_2
y = data_new['Star Rating_encoded']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# splitting the data into test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 100)

In [174]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier

In [175]:
lrcv = LogisticRegressionCV()
gnb = GaussianNB()
random_forest = RandomForestClassifier(criterion='entropy',
                                      min_samples_leaf=5)
dt = DecisionTreeClassifier()

In [176]:
lrcv.fit(X_train, y_train)
gnb.fit(X_train, y_train)
random_forest.fit(X_train,y_train)
dt.fit(X_train, y_train)

/home/deepesh/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/home/deepesh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/deepesh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/deepesh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/deepesh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarn

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [177]:
y_lrcv_pred = lrcv.predict(X_test)
y_gnb_pred = gnb.predict(X_test)
random_forest_pred = random_forest.predict(X_test)
dt_pred = dt.predict(X_test)

In [178]:
from sklearn.metrics import confusion_matrix, f1_score

In [189]:
lrcv_cm = confusion_matrix(y_test, y_lrcv_pred)
gnb_cm = confusion_matrix(y_test, y_gnb_pred)
random_forest_cm = confusion_matrix(y_test, random_forest_pred)
dt_cm = confusion_matrix(y_test, dt_pred)
print(dt.score(X_test, y_test))
print(random_forest.score(X_test, y_test))
print(gnb.score(X_test, y_test))
print(lrcv.score(X_test, y_test))

0.858647936786655
0.8823529411764706
0.8287971905179983
0.8990342405618964


In [191]:
X_test['predicted'] = random_forest_pred

/home/deepesh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [210]:
data_new_high = data_new[data_new['Star Rating'] > 3]
data_new_high = data_new_high.reset_index()
data_new_low = data_new[data_new['Star Rating'] < 4 ]
data_new_low = data_new_low.reset_index()

In [211]:
BOW_high = []
for rev in data_new_high['Review Text']:
    for word in rev:
        if word not in BOW_high:
            BOW_high.append(word)

In [212]:
BOW_low = []
for rev in data_new_low['Review Text']:
    for word in rev:
        if word not in BOW_low:
            BOW_low.append(word)

In [213]:
bow_df_high = pd.DataFrame(columns=BOW_high, index=data_new_high['id'])
bow_df_high = bow_df_high.fillna(0)

In [214]:
bow_df_low = pd.DataFrame(columns=BOW_low, index=data_new_low['id'])
bow_df_low = bow_df_low.fillna(0)

In [216]:
for i, doc in enumerate(data_new_high['Review Text']):
    for c in bow_df_high.columns:
        if c in doc:
            bow_df_high[c][i] = 1

In [217]:
for i, doc in enumerate(data_new_low['Review Text']):
    for c in bow_df_low.columns:
        if c in doc:
            bow_df_low[c][i] = 1

In [218]:
bow_df_low.shape, bow_df_high.shape

((2158, 3428), (3534, 2263))

In [219]:
for c in bow_df_high.columns:
    if bow_df_high[c].sum() < 50:
        bow_df_high = bow_df_high.drop(c, axis = 1)

In [220]:
for c in bow_df_low.columns:
    if bow_df_low[c].sum() < 50:
        bow_df_low = bow_df_low.drop(c, axis = 1)

In [221]:
bow_df_low.shape, bow_df_high.shape

((2158, 140), (3534, 76))

In [222]:
X_high = bow_df_high
y_high = data_new_high['Star Rating']
X_low = bow_df_low
y_low = data_new_low['Star Rating']

In [223]:
X_train_high, X_test_high, y_train_high, y_test_high = train_test_split(X_high, y_high, test_size = 0.1)
X_train_low, X_test_low, y_train_low, y_test_low = train_test_split(X_low, y_low, test_size = 0.1)


In [224]:
rf_high = RandomForestClassifier()
rf_low = RandomForestClassifier()

In [225]:
rf_high.fit(X_train_high, y_train_high)
rf_low.fit(X_train_low, y_train_low)

/home/deepesh/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/deepesh/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)